# 02 – Plugins
Extend your agent with account lookup and plan details plugins.

In [ ]:
import uuid
from typing import Annotated, Dict
from pydantic import BaseModel
from semantic_kernel.functions import kernel_function

# Mock database for phone number to UUID mapping
phone_to_uuid: Dict[str, str] = {}

class AccountLookupPlugin:
    @kernel_function(description="Maps a phone number to a customer UUID.")
    def get_customer_uuid(
        self, phone_number: Annotated[str, "The customer's phone number."]
    ) -> Annotated[str, "Returns the UUID for the customer."]:
        if phone_number not in phone_to_uuid:
            phone_to_uuid[phone_number] = str(uuid.uuid4())
        return phone_to_uuid[phone_number]

class CustomerPlanDetails(BaseModel):
    uuid: str
    plan_name: str
    data_limit_gb: int
    unlimited_text: bool
    monthly_cost: float

# Mock database for UUID to plan details mapping
uuid_to_plan: Dict[str, CustomerPlanDetails] = {}

class PlanDetailsPlugin:
    @kernel_function(description="Retrieves mocked phone plan details for a customer UUID.")
    def get_plan_details(
        self, customer_uuid: Annotated[str, "The customer's UUID."]
    ) -> Annotated[str, "Returns the customer's phone plan details."]:
        if customer_uuid not in uuid_to_plan:
            # Mock some plan details
            uuid_to_plan[customer_uuid] = CustomerPlanDetails(
                uuid=customer_uuid,
                plan_name="Unlimited Plus",
                data_limit_gb=50,
                unlimited_text=True,
                monthly_cost=79.99
            )
        details = uuid_to_plan[customer_uuid]
        return (
            f"Plan: {details.plan_name}\n"
            f"Data Limit: {details.data_limit_gb}GB\n"
            f"Unlimited Text: {'Yes' if details.unlimited_text else 'No'}\n"
            f"Monthly Cost: ${details.monthly_cost:.2f}"
        )

In [ ]:
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

agent = ChatCompletionAgent(
    service=AzureChatCompletion(),
    name="CustomerServiceAgent",
    instructions="Use plugins to lookup account and plan.",
    plugins=[AccountLookupPlugin(), PlanDetailsPlugin()]
)

In [ ]:
response = await agent.get_response(messages="What plan do I have? (555) 123‑4567")
print(response.content)